In [18]:
from itertools import product
from random import shuffle, randint

In [ ]:
def generate_grid_smart(rows, columns, npits, nwalls):
    tiles = list(product(range(rows), (range(columns))))
    tile_type = {tile:"ice" for tile in tiles}
    shuffle(tiles)

    init = tiles.pop()
    goal = tiles.pop()

    nwaypoints = randint(2,3)
    waypoints = [tiles.pop() for i in range(nwaypoints)]
    current = init
    for wp in waypoints:
        xdist = wp[0] - current[0]
        ydist = wp[1] - current[1]


    pits = [tiles.pop() for i in range(npits)]
    for pit in pits:
        tile_type[pit] = "pit"
    walls = [tiles.pop() for i in range(nwalls)]
    for wall in walls:
        tile_type[wall] = "wall"

    return init, goal, tile_type

In [2]:
def generate_grid(rows, columns, npits, nwalls):
    tiles = list(product(range(rows), (range(columns))))
    tile_type = {tile:"ice" for tile in tiles}
    shuffle(tiles)

    init = tiles.pop()
    goal = tiles.pop()
    pits = [tiles.pop() for i in range(npits)]
    for pit in pits:
        tile_type[pit] = "pit"
    walls = [tiles.pop() for i in range(nwalls)]
    for wall in walls:
        tile_type[wall] = "wall"

    return init, goal, tile_type

In [10]:
def generate_objects_string(rows, columns, tile_type):
    obj_str = "(:objects \n"
    tile_list = ["upperwall-{} - wall".format(x) for x in range(columns)]
    obj_str += " ".join(tile_list) + "\n"
    for y in range(rows):
        tile_list = []
        tile_list += ["leftwall-{} - wall".format(rows-y-1)]
        for x in range(columns):
            coord = (x, rows-y-1)
            tile_list += ["tile_{}_{} - {}".format(coord[0],coord[1], tile_type[coord])]
        tile_list += ["rightwall-{} - wall".format(rows-y-1)]
        obj_str += " ".join(tile_list) + "\n"
    tile_list = ["bottomwall-{} - wall".format(x) for x in range(columns)]
    obj_str += " ".join(tile_list) + "\n"
    obj_str += ")\n"

    return obj_str


In [11]:
def generate_init_string(rows, columns, init):
    init_str = "(:init\n"
    init_str += "(at tile_{}_{})\n".format(init[0], init[1])
    init_str += "\n"
    
    for y in range(rows):
        tile_list = []
        tile_list += ["(left-of leftwall-{} tile_{}_{})".format(rows-y-1,0,rows-y-1)]
        for x in range(columns-1):
            tile_list += ["(left-of tile_{}_{} tile_{}_{})".format(x,rows-y-1, x+1,rows-y-1)]
        tile_list += ["(left-of tile_{}_{} rightwall-{})".format(columns-1,rows-y-1,rows-y-1)]
        init_str += " ".join(tile_list) + "\n"
    init_str += "\n"

    init_str += " ".join(["(down-of tile_{}_{} upperwall-{})".format(i,rows-1, i) for i in range(columns)] + ["\n"])
    for y in range(rows-1):
        tile_list = []
        for x in range(columns):
            tile_list += ["(down-of tile_{}_{} tile_{}_{})".format(x,rows-y-2, x,rows-y-1)]
        init_str += " ".join(tile_list) + "\n"
    init_str += " ".join(["(down-of bottomwall-{} tile_{}_{})".format(i, i,0) for i in range(columns)])
    init_str += ")\n"

    return init_str

In [12]:
def generate_goal_string(goal):
    goal_str = "(:goal\n"
    goal_str += "(and (at tile_{}_{}))\n".format(goal[0], goal[1])
    goal_str += ")\n"

    return goal_str

In [15]:
def generate_problem_string(pnum, rows, columns, init, goal, tile_type):
    p_str = ""
    p_str += "(define (problem p{}) (:domain icylake)\n".format(str(pnum).zfill(2))
    p_str += "\n"

    p_str += generate_objects_string(rows,columns,tile_type)
    p_str += "\n"

    p_str += generate_init_string(rows,columns,init)
    p_str += "\n"

    p_str += generate_goal_string(goal)
    p_str += ")"

    return p_str


In [17]:
rows = 6
columns = 6

pnum = 0
npits = columns - 2
nwalls = columns - 4

init, goal, tile_type = generate_grid(rows, columns, npits, nwalls)
problem = generate_problem_string(pnum, rows, columns, init, goal, tile_type)

# print(problem)

with open("benchmarks/frozenlake/p{}.pddl".format(str(pnum).zfill(2)), "w") as f:
    f.write(problem)


In [20]:
sizes = [6]
NINSTANCES = 99

pnum = 0
for size in sizes:
    rows = columns = size
    for i in range(NINSTANCES):
        pnum += 1
        npits = randint(1, size-1)
        nwalls = randint(1, size-1)

        init, goal, tile_type = generate_grid(rows, columns, npits, nwalls)
        problem = generate_problem_string(pnum, rows, columns, init, goal, tile_type)

        # print(problem)

        with open("benchmarks/frozenlake/p{}.pddl".format(str(pnum).zfill(2)), "w") as f:
            f.write(problem)

In [ ]:


tile_str = ""
tile_list = ["upperwall-{} - wall".format(x) for x in range(columns)]
tile_str += " ".join(tile_list) + "\n"
for y in range(rows):
    tile_list = []
    tile_list += ["leftwall-{} - wall".format(rows-y-1)]
    for x in range(columns):
        coord = (x, rows-y-1)
        tile_list += ["tile_{}_{} - {}".format(coord[0],coord[1], tile_type[coord])]
    tile_list += ["rightwall-{} - wall".format(rows-y-1)]
    tile_str += " ".join(tile_list) + "\n"
tile_list = ["bottomwall-{} - wall".format(x) for x in range(columns)]
tile_str += " ".join(tile_list) + "\n"
print(tile_str)

In [ ]:
rows = 6
columns = 6
tile_str = ""
for y in range(rows):
    tile_list = []
    tile_list += ["(left-of leftwall-{} tile_{}_{})".format(rows-y-1,0,rows-y-1)]
    for x in range(columns-1):
        tile_list += ["(left-of tile_{}_{} tile_{}_{})".format(x,rows-y-1, x+1,rows-y-1)]
    tile_list += ["(left-of tile_{}_{} rightwall-{})".format(columns-1,rows-y-1,rows-y-1)]
    tile_str += " ".join(tile_list) + "\n"
print(tile_str)

In [ ]:
rows = 6
columns = 6

tile_str = ""
tile_str += " ".join(["(down-of tile_{}_{} upperwall-{})".format(i,rows-1, i) for i in range(columns)] + ["\n"])
for y in range(rows-1):
    tile_list = []
    for x in range(columns):
        tile_list += ["(down-of tile_{}_{} tile_{}_{})".format(x,rows-y-2, x,rows-y-1)]
    tile_str += " ".join(tile_list) + "\n"
tile_str += " ".join(["(down-of bottomwall-{} tile_{}_{})".format(i, i,0) for i in range(columns)])
print(tile_str)